In [1]:
from dotenv import load_dotenv
load_dotenv("./.env")

from langchain.tools import tool
from langchain.chat_models import ChatOpenAI
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner

In [2]:
# 库存查询
@tool
def check_inventory(flower_type: str) -> int:
    """
    查询特定类型花的库存数量。
    参数:
    - flower_type: 花的类型
    返回:
    - 库存数量 (暂时返回一个固定的数字)
    """
    # 实际应用中这里应该是数据库查询或其他形式的库存检查
    return 100  # 假设每种花都有100个单位

# 定价函数
@tool
def calculate_price(base_price: float, markup: float) -> float:
    """
    根据基础价格和加价百分比计算最终价格。
    参数:
    - base_price: 基础价格
    - markup: 加价百分比
    返回:
    - 最终价格
    """
    return base_price * (1 + markup)

# 调度函数
@tool
def schedule_delivery(order_id: int, delivery_date: str):
    """
    安排订单的配送。
    参数:
    - order_id: 订单编号
    - delivery_date: 配送日期
    返回:
    - 配送状态或确认信息
    """
    # 在实际应用中这里应该是对接配送系统的过程
    return f"订单 {order_id} 已安排在 {delivery_date} 配送"

tools = [check_inventory, calculate_price]
tools

[StructuredTool(name='check_inventory', description='查询特定类型花的库存数量。\n参数:\n- flower_type: 花的类型\n返回:\n- 库存数量 (暂时返回一个固定的数字)', args_schema=<class 'pydantic.v1.main.check_inventorySchema'>, func=<function check_inventory at 0x7fd013f6f490>),
 StructuredTool(name='calculate_price', description='根据基础价格和加价百分比计算最终价格。\n参数:\n- base_price: 基础价格\n- markup: 加价百分比\n返回:\n- 最终价格', args_schema=<class 'pydantic.v1.main.calculate_priceSchema'>, func=<function calculate_price at 0x7fd012147ac0>)]

In [3]:
model = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)


/Users/wenke/.conda/envs/agent/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [4]:
# 设置计划者和执行者
planner = load_chat_planner(model)
executor = load_agent_executor(model, tools, verbose=True)

# 初始化Plan-and-Execute Agent
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)


In [5]:
# 运行Agent解决新问题（完善了需求）
agent.run("查查玫瑰的库存然后给出50朵玫瑰的价格和当天的配送方案！")

/Users/wenke/.conda/envs/agent/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new PlanAndExecute chain...
steps=[Step(value='查询玫瑰的库存情况。'), Step(value='如果有足够的50朵玫瑰，则获取50朵玫瑰的价格。'), Step(value='查询当天的配送方案。'), Step(value='给出50朵玫瑰的价格和当天的配送方案。'), Step(value='结束。\n\n')]

> Entering new AgentExecutor chain...
Thought: I should use the check_inventory tool to find out the inventory quantity of roses.

Action:
```
{
  "action": "check_inventory",
  "action_input": {"flower_type": "玫瑰"}
}
```
Observation: 100
Thought:Since the inventory quantity of roses is 100, I can provide this information to the user as the final answer.
Action:
```
{
  "action": "Final Answer",
  "action_input": "The inventory quantity of roses is 100."
}
```

> Finished chain.
*****

Step: 查询玫瑰的库存情况。

Response: The inventory quantity of roses is 100.

> Entering new AgentExecutor chain...
Thought: I should check the inventory for roses to see if there are enough in stock to provide the price for 50 roses.

Action:
```
{
  "action": "check_inventory",
  "action_input": {"flower_type": "ros

'Thank you for using the flower shop service. Have a great day!'